In [ ]:
import networkx as nx
import numpy as np
import seaborn as sb
import scipy
import matplotlib.pyplot as plt
import cvxpy as cp

# Methods

In [2]:
def sup_lower_approx(VV, n):
    return np.ceil(1/np.max(VV))

In [3]:
def lower_approx(VV, n):
    vflat = VV.reshape(n*n)
    res = 0
    count = 0
    while res < 1:
        count += 1
        res += vflat[np.argsort(vflat)[-count]]
    return count

In [4]:
def linear_approx(V2, n):
    x = cp.Variable(n)
    y = cp.Variable(n)
    u = cp.Variable((n,n))

    c1 = cp.sum(cp.multiply(V2, u))
    cx = cp.reshape(x,(n,1)) @ cp.reshape(np.ones([1,n]),(1,n))
    cy = cp.reshape(np.ones([1,n]),(n,1)) @ cp.reshape(y,(1,n))

    constraints = [cp.vec(u) >= 0, cp.vec(u) <= 1, 0 <= x, x <= 1, 0 <= y, y <= 1, c1 >= 1, u <= cx, u <= cy, cx + cy - 1  <= u]
    prob = cp.Problem(cp.Minimize(cp.sum(u)),constraints)
    return np.ceil(prob.solve())

In [5]:
def sdp_approx(VV, n):
    x = cp.Variable(2*n)
    u = cp.Variable((2*n,2*n))
    
    Z = np.zeros((n,n))
    O = np.ones((n,n))
    Vsym = np.concatenate((np.concatenate((Z,0.5*VV), axis =1), np.concatenate((0.5*VV.T, Z), axis =1)))
    Idsym = np.concatenate((np.concatenate((Z,0.5*O), axis =1), np.concatenate((0.5*O, Z), axis =1)))

    c1 = cp.sum(cp.multiply(Vsym, u))
    cx = cp.reshape(x,(2*n,1)) @ cp.reshape(np.ones([1,2*n]),(1,2*n))
    cy = cp.reshape(np.ones([1,2*n]),(2*n,1)) @ cp.reshape(x,(1,2*n))
    ctrace = cp.trace(u - cp.diag(x))
    
    A11= cp.reshape(1, (1, 1))
    A12= cp.reshape(x, (1, 2*n))
    A1 = cp.hstack([A11, A12])

    A21= cp.reshape(x, (2*n,1))
    A22= u
    A2 = cp.hstack([A21, A22])
    M = cp.vstack([A1, A2])
    

    constraints = [cp.vec(u) >= 0, cp.vec(u) <= 1, 0 <= x, x <= 1, c1 >= 1, u <= cx, u <= cy, cx + cy - 1  <= u, ctrace == 0, M >> 0]
    prob = cp.Problem(cp.Minimize(cp.sum(cp.multiply(Idsym, u))), constraints)
    return np.ceil(prob.solve())

In [6]:
def upper_bound_3(VV, n):
    res = []
    for i in range(n):
        print('working...', i)
        for j in range(i+1,n):
            for t in range(j+1, n):
                ind_subset = [i,j,t]
                VV_tr = VV[ind_subset, :]
                sums = np.sum(VV_tr, axis = 0)
                argsorted_sums = np.argsort(sums)
                cumsum = 0
                count = 0
                while cumsum < 1:
                    count += 1
                    cumsum += sums[argsorted_sums[-count]]
                res.append(count*3)
                VV_tr = VV[:,ind_subset]
                sums = np.sum(VV_tr, axis = 1)
                argsorted_sums = np.argsort(sums)
                cumsum = 0
                count = 0
                while cumsum < 1:
                    count += 1
                    cumsum += sums[argsorted_sums[-count]]
                res.append(count*3)
    return np.min(res)

In [7]:
def upper_bound_2(VV, n):
    res = []
    for i in range(n):
        for j in range(i+1,n):
            ind_subset = [i,j]
            VV_tr = VV[ind_subset, :]
            sums = np.sum(VV_tr, axis = 0)
            argsorted_sums = np.argsort(sums)
            cumsum = 0
            count = 0
            while cumsum < 1:
                count += 1
                cumsum += sums[argsorted_sums[-count]]
            res.append(count*2)
            VV_tr = VV[:,ind_subset]
            sums = np.sum(VV_tr, axis = 1)
            argsorted_sums = np.argsort(sums)
            cumsum = 0
            count = 0
            while cumsum < 1:
                count += 1
                cumsum += sums[argsorted_sums[-count]]
            res.append(count*2)
    return np.min(res)

# Models

In [9]:
def random_regular(n, d):
    g = nx.random_regular_graph(d, n, seed=None)
    matrix = nx.to_numpy_matrix(g)
    u, V = np.linalg.eigh(matrix)
    VV = np.array(np.multiply(V, V))
    return VV

In [8]:
def random_regular_modified(n, d, k1, k2):
    g = nx.random_regular_graph(d, n, seed=None)
    mat = nx.to_numpy_matrix(g)
    mat_modified = modify_adjacency(mat.copy(), n, k1, k2)
    u, V = np.linalg.eigh(mat_modified)
    VV = np.array(np.multiply(V, V))
    return VV

In [ ]:
saves = []
for n in range(25,475,25):
    VV = random_regular(n, 2*int(np.ceil(np.sqrt(n))))
    saves.append(VV)
with open('test-adjacencies1.pkl','wb') as f:
     pickle.dump(saves, f)

In [ ]:
matrices = saves
#matrices = pickle.load(open("test-adjacencies.pkl", "rb"))
tries = num_tries

In [ ]:
tries = num_tries

linear_lowers = []
sdp_lowers = []
greedy_lowers = []
sup_lowers = []
two_uppers = []
three_uppers = []

for _ in range(tries):
    ll = []
    sl = []
    gl = []
    sdpl = []
    tu = []
    tru = []
    for i in range(13, 14):
        VV = matrices[i]
        n = VV.shape[0]
        print(n)
        sdpl.append(sdp_approx(VV, n))
        print(sdpl)
        print('sdp done')
    
    
        ll.append(linear_approx(VV, n))
        print('linear done')
        gl.append(lower_approx(VV, n))
        print('greedy done')

        sl.append(sup_lower_approx(VV, n))
        print('sup done')


        x = upper_bound_2(VV, n)
        tu.append(x)
        print('two upper done')
        y = upper_bound_3(VV, n)
        print('three upper done')
        tru.append(y)
        
    print(sdpl)
    sdp_lowers.append(sdpl)
    print(ll)
    linear_lowers.append(ll)
    print(gl)
    greedy_lowers.append(gl)
    print(sl)
    sup_lowers.append(sl)
    print(tu)
    two_uppers.append(tu)
    print(tru)
    three_uppers.append(tru)
        
